In [22]:
import json
import ast
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import folium
from folium.plugins import MarkerCluster

In [23]:
def add_lat_long(df):
    loc_array = df['location']
    
    loc_list= []
    for i in loc_array:
        loc_list.append(ast.literal_eval(i))
    
    new_df = pd.concat([df, pd.DataFrame(loc_list)], axis=1)
    return new_df

In [104]:
df = pd.read_csv('bird_data.csv')
df.drop(['code', 'captive'], axis=1, inplace=True)

temp_df = df.head(10000)

new_df = add_lat_long(temp_df)
new_df['coordinates'] = list(zip(new_df.latitude, new_df.longitude))
new_df['count'] = 1
new_df['grid_location'] = 0

/Users/johnhayes/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
def get_geojson_grid(upper_right, lower_left, n=6):
    """Returns a grid of geojson rectangles, and computes the exposure in each section of the grid based on the vessel data.

    Parameters
    ----------
    upper_right: array_like
        The upper right hand corner of "grid of grids" (the default is the upper right hand [lat, lon] of the USA).

    lower_left: array_like
        The lower left hand corner of "grid of grids"  (the default is the lower left hand [lat, lon] of the USA).

    n: integer
        The number of rows/columns in the (n,n) grid.

    Returns
    -------

    list
        List of "geojson style" dictionary objects   
    """

    all_boxes = []

    lat_steps = np.linspace(lower_left[0], upper_right[0], n+1)
    lon_steps = np.linspace(lower_left[1], upper_right[1], n+1)

    lat_stride = lat_steps[1] - lat_steps[0]
    lon_stride = lon_steps[1] - lon_steps[0]

    for lat in lat_steps[:-1]:
        for lon in lon_steps[:-1]:
            # Define dimensions of box in grid
            upper_left = [lon, lat + lat_stride]
            upper_right = [lon + lon_stride, lat + lat_stride]
            lower_right = [lon + lon_stride, lat]
            lower_left = [lon, lat]

            # Define json coordinates for polygon
            coordinates = [
                upper_left,
                upper_right,
                lower_right,
                lower_left,
                upper_left
            ]

            geo_json = {"type": "FeatureCollection",
                        "properties":{
                            "lower_left": lower_left,
                            "upper_right": upper_right
                        },
                        "features":[]}

            grid_feature = {
                "type":"Feature",
                "geometry":{
                    "type":"Polygon",
                    "coordinates": [coordinates],
                }
            }

            geo_json["features"].append(grid_feature)

            all_boxes.append(geo_json)

    return all_boxes

NameError: name 'accident_data' is not defined

In [18]:
lower_left = [37.808136-.04, -122.256303-.04]
upper_right = [37.808136+.04, -122.256303+.04]
m = folium.Map(zoom_start = 13, location=[37.808136, -122.256303])

grid = get_geojson_grid(upper_right, lower_left , n=10)


for i, geo_json in enumerate(grid):

    color = plt.cm.Reds(i / len(grid))
    color = mpl.colors.to_hex(color)

    gj = folium.GeoJson(geo_json,
                        style_function=lambda feature, color=color: {
                                                                        'fillColor': color,
                                                                        'color':"black",
                                                                        'weight': 2,
                                                                        'dashArray': '5, 5',
                                                                        'fillOpacity': 0.55,
                                                                    })
    popup = folium.Popup("Grid Square: {}".format(i))
    gj.add_child(popup)

    m.add_child(gj)
m

In [25]:
m = folium.Map(location=[37.808136, -122.256303], zoom_start=13)

locations = list(zip(new_df.latitude, new_df.longitude))
icons = [folium.Icon(icon="dove", prefix="fa") for _ in range(len(locations))]

cluster = MarkerCluster(locations=locations, icons=icons)
m.add_child(cluster)
m

In [89]:
m = folium.Map(zoom_start = 13, location=[37.808136, -122.256303])

# Generate GeoJson grid
top_right = [new_df['latitude'].max(), new_df['longitude'].max()]
top_left = [new_df['latitude'].min(), new_df['longitude'].min()]


# consider changing n to 64 for more specific results
grid = get_geojson_grid(top_right, top_left, n=42)

# Calculate exposures in grid
popups = []
regional_counts = []
total_on_map = 0
for i, box in enumerate(grid):
    upper_right = box["properties"]["upper_right"]
    lower_left = box["properties"]["lower_left"]

    mask = (
            (new_df.latitude <= upper_right[1]) & (new_df.latitude >= lower_left[1]) &
            (new_df.longitude <= upper_right[0]) & (new_df.longitude >= lower_left[0])
           )

    region_incidents = len(new_df[mask])
    regional_counts.append(region_incidents)

    total_vehicles = new_df[mask]['count'].sum()
    #total_casualties = accident_data[mask].Number_of_Casualties.sum()
    content = "total scooties {:,.0f}".format(total_vehicles)
    popup = folium.Popup(content)
    popups.append(popup)
    
    column_name = 'grid_location'
    new_df.loc[mask, column_name] = i
    
    total_on_map += total_vehicles
worst_region = max(regional_counts)

# Add GeoJson to map
for i, box in enumerate(grid):
    geo_json = json.dumps(box)

    color = plt.cm.Reds(regional_counts[i] / worst_region)
    color = mpl.colors.to_hex(color)

    gj = folium.GeoJson(geo_json,
                        style_function=lambda feature, color=color: {
                                                                        'fillColor': color,
                                                                        'color':"black",
                                                                        'weight': .5,
                                                                        'dashArray': '5, 5',
                                                                        'fillOpacity': 0.55,
                                                                    })
    gj.add_child(popups[i])
    m.add_child(gj)
    
# Marker clusters
locations = list(zip(new_df.latitude, new_df.longitude))
icons = [folium.Icon(icon="car", prefix="fa") for _ in range(len(locations))]

# Create popups
popup_content = []
# for incident in accident_data.itertuples():
#     number_of_vehicles = "Number of vehicles: {} ".format(incident.Number_of_Vehicles)
#     number_of_casualties = "Number of casualties: {}".format(incident.Number_of_Casualties)
#     content = number_of_vehicles + number_of_casualties
#     popup_content.append(content)

popups = [folium.Popup(content) for content in popup_content]

cluster = MarkerCluster(locations=locations, icons=icons, popups=popups)
m.add_child(cluster)

m.save("scooters.html")
print (total_on_map)

10000


1000

In [88]:
print(new_df['latitude'].max(), new_df['latitude'].min(), new_df['longitude'].max(), new_df['longitude'].min())

37.872345 37.75016 -122.16694666666666 -122.30040666666667


In [62]:
folium.GeoJson?

Init signature: folium.GeoJson(data, style_function=None, name=None, overlay=True, control=True, show=True, smooth_factor=None, highlight_function=None, tooltip=None)
Docstring:     
Creates a GeoJson object for plotting into a Map.

Parameters
----------
data: file, dict or str.
    The GeoJSON data you want to plot.
    * If file, then data will be read in the file and fully
    embedded in Leaflet's JavaScript.
    * If dict, then data will be converted to JSON and embedded
    in the JavaScript.
    * If str, then data will be passed to the JavaScript as-is.
style_function: function, default None
    Function mapping a GeoJson Feature to a style dict.
highlight_function: function, default None
    Function mapping a GeoJson Feature to a style dict for mouse events.
name : string, default None
    The name of the Layer, as it will appear in LayerControls
overlay : bool, default True
    Adds the layer as an optional overlay (True) or the base layer (False).
control : bool, default T

In [90]:
new_df.head()

,battery_level,id,location,time,latitude,longitude,coordinates,count,grid_location
0,44,cd6ff74b-cba6-49fe-b115-cec6066fe919,"{'latitude': 37.845245, 'longitude': -122.2935...",2018-11-09 00:55:30.991958,37.845245,-122.293503,"(37.845245, -122.29350333333332)",1,1346
1,48,e20d8a6c-2453-494a-a083-00bad445b189,"{'latitude': 37.846000000000004, 'longitude': ...",2018-11-09 00:55:30.991958,37.846000,-122.295692,"(37.846000000000004, -122.29569166666667)",1,1345
2,48,c8f9bed6-48bd-405d-a290-84bd0148a700,"{'latitude': 37.84542833333333, 'longitude': -...",2018-11-09 00:55:30.991958,37.845428,-122.288238,"(37.84542833333333, -122.28823833333333)",1,1347
3,38,b7ac0ffe-77dd-40dc-82f1-cb59e0f6fb6b,"{'latitude': 37.84618, 'longitude': -122.28023...",2018-11-09 00:55:30.991958,37.846180,-122.280235,"(37.84618, -122.28023499999999)",1,1392
4,81,7e9d6f7f-90ba-4205-b040-1c5025512467,"{'latitude': 37.829296666666664, 'longitude': ...",2018-11-09 00:55:30.991958,37.829297,-122.280830,"(37.829296666666664, -122.28083)",1,1140


In [102]:
def add_time_chunk(df):
    df['time_chunk'] = 0
    for i, time in enumerate(df['time'].unique()):
        df['time_chunk'][df['time'] == time] = i
    return df

In [106]:
time_chunk_df = add_time_chunk(new_df)

/Users/johnhayes/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [109]:
print (len(new_df['time'].unique()), len(time_chunk_df['time_chunk'].unique()))

40 40
